<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.883 · Aprenentatge per reforç</p>
<p style="margin: 0; text-align:right;">Màster universitari de Ciència de Dades</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Mòdul 9: Exemple de DQN en l'entorn CartPole


En aquest _notebook_ veurem un exemple d'implementació d'una Deep Q-Network (DQN) utilitzant un entorn ja predefinit en OpenAI.

Tant per a aquest exemple com per a les pràctiques posteriors s'utilitzarà el <i>framework</i> de __Pytorch__. 

## 1. Entorn CartPole

En aquest exemple usarem un entorn ja definit en la llibreria d'OpenAI, però cal tenir present que en altres problemes més concrets l'entorn necessitarà ser definit.

CartPole consisteix a aprendre a controlar un objecte. El joc consta d'un carretó i d'un pal col·locat verticalment damunt del carretó. El pal s'aguanta únicament per la gravetat, mentre que el carretó es mou a dreta i esquerra sense parar. L'objectiu de l'agent és controlar la velocitat del carretó augmentant-la o disminuint-la amb la condició d'evitar que el pal caigui.

### 1.1. Establiment de l'entorn

En primer lloc, carregarem la llibreria __gym__ i inicialitzarem l'entorn.

In [1]:
import gym

env = gym.envs.make("CartPole-v0")

Cada entorn té definit tot el necessari perquè un agent pugui aprendre: tenim un joc que funciona d'una manera determinada i podem entrenar un agent perquè aprengui a jugar a aquest joc sense cap més ajuda que la d'experimentar-hi observant, actuant i rebent recompenses. Així, l'entorn del joc ja defineix quines accions es poden prendre, quines situacions poden presentar-se, en què consistirà la recompensa, etc. 

A continuació, podem visualitzar l'entorn de __CartPole__ generant un bucle sobre uns pocs episodis i, en acabar, el tanquem. 

In [2]:
#Visualitzem l'entorn
for i_episode in range(15):
    observation = env.reset()
    for t in range(100):
        #env.render()  #EL RENDER NOMÉS FUNCIONA EN LOCAL: comentar línia si no s'està en local.
        print(observation)
        action = env.action_space.sample() #acció aleatòria
        observation, reward, done, info = env.step(action) #execució de l'acció triada
        if done:
            print("Episode finished after {} timesteps".format(t+1)) 
            break

env.close() #tanquem la visualització de l'entorn

[-0.02148017 -0.04427617 -0.04407625 -0.00787081]
[-0.02236569 -0.2387392  -0.04423367  0.27058622]
[-0.02714048 -0.43320294 -0.03882194  0.54899605]
[-0.03580453 -0.23755776 -0.02784202  0.24433862]
[-0.04055569 -0.04204943 -0.02295525 -0.05699472]
[-0.04139668 -0.23683485 -0.02409514  0.22835818]
[-0.04613337 -0.43160433 -0.01952798  0.51334431]
[-0.05476546 -0.62644587 -0.00926109  0.79981005]
[-0.06729438 -0.82143957  0.00673511  1.08956533]
[-0.08372317 -0.62640705  0.02852642  0.79900329]
[-0.09625131 -0.82190849  0.04450648  1.10052177]
[-0.11268948 -1.01758697  0.06651692  1.40682942]
[-0.13304122 -1.2134684   0.09465351  1.71954335]
[-0.15731059 -1.40953893  0.12904437  2.04011924]
[-0.18550137 -1.21595922  0.16984676  1.78999783]
[-0.20982055 -1.4125311   0.20564671  2.13030807]
Episode finished after 16 timesteps
[-0.02262192 -0.02671578 -0.00931504  0.03742169]
[-0.02315624 -0.22170292 -0.00856661  0.32715113]
[-0.0275903  -0.41670187 -0.00202359  0.61712027]
[-0.03592433 -

La recompensa és 1 per cada pas donat, inclòs l'estat terminal. Es considera l'entorn resolt quan la mitjana de les recompenses és major o igual a 195.0 després de 100 intents consecutius.

## 2. Construcció d'una DQN: ensenyar un agent a jugar

La construcció d'una DQN per ensenyar un agent a jugar a **CartPole** té, com hem vist en el mòdul didàctic, els passos següents:

<ol>
    <li> Definir el model de xarxa neuronal. </li>
    <li> Definir l'agent: com s'ha de comportar, quan ha de seleccionar una acció i com. </li>
    <li> Fixar hiperparàmetres. </li>
    <li> Entrenar l'agent.  </li>
    
</ol>


Començarem important la llibreria per treballar en **Pytorch** i altres llibreries necessàries:

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

### 2.1. Definició del model

El primer pas és definir la nostra xarxa neuronal, la DQN. Per a aquest exemple, usarem una xarxa neuronal molt senzilla amb tres capes lineals i dues capes ReLU, a més de l'optimizador Adam.

També indicarem la possibilitat de treballar amb **CPU** o **CUDA** per si es té l'opció, ja que en aprenentatge per reforç la majoria dels processos solen requerir molta màquina, i l'acceleració per maquinari és normalment necessària. **Aquest exemple es pot executar amb CPU**.


Com s'explicava en el mòdul teòric, perquè l'aprenentatge prosperi és important que les aproximacions de _Q_ siguin prou bones perquè les experiències aportin informació rellevant a l'agent. Si no s'aconsegueixen bons valors, l'agent corre el risc d'estancar-se entre decisions dolentes sense mostrar cap millora. Per a això s'introdueix el **mètode <i>e-greedy</i>**, que permet a l'agent explorar accions aleatòries durant un temps a l'inici de l'entrenament i facilita que vagi passant a utilitzar l'aproximació de _Q_ a poc a poc (explotació). Recordem que aquest comportament ve definit per l'hiperparàmetre de probabilitat <i>epsilon</i>.

In [4]:
class DQN(torch.nn.Module):
    
    def __init__(self, env, learning_rate=1e-3, device='cpu'):
        super(DQN, self).__init__()
        self.device = device
        self.n_inputs = env.observation_space.shape[0]
        self.n_outputs = env.action_space.n
        self.actions = np.arange(env.action_space.n)
        self.learning_rate = learning_rate
        
        ### Construcció de la xarxa neuronal
        self.model = torch.nn.Sequential(
            torch.nn.Linear(self.n_inputs, 16, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 16, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(16, self.n_outputs, bias=True))
        
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        
        
        ### S'ofereix l'opció de treballar amb CUDA
        if self.device == 'cuda':
            self.model.cuda()
            
    
    ### Mètode e-greedy
    def get_action(self, state, epsilon=0.05):
        if np.random.random() < epsilon:
            action = np.random.choice(self.actions)  # acció aleatòria
        else:
            qvals = self.get_qvals(state)  # acció a partir del càlcul del valor de Q per a aquesta acció
            action= torch.max(qvals, dim=-1)[1].item()
        return action
    
    
    def get_qvals(self, state):
        if type(state) is tuple:
            state = np.array([np.ravel(s) for s in state])
        state_t = torch.FloatTensor(state).to(device=self.device)
        return self.model(state_t)

#### 2.1.1. _Buffer_ de repetició d'experiències

Un altre problema de l'algorisme bàsic de la DQN era la seqüencialitat de les dades: els estats estan molt correlacionats i la xarxa neuronal no pot funcionar bé amb tanta correlació. Introduint un **<i>buffer</i> de repetició d'experiències**, permetem que s'emmagatzemin unes quantes experiències passades i que es passi un subconjunt aleatori d'aquestes experiències a la xarxa neuronal. Al seu torn, el *buffer* s'ha d'anar alimentant d'experiències noves conforme l'agent va aprenent.

Primer importem les funcions `deque` i `namedtuple` de la llibreria `collections`. El *deque* és un objecte que emmagatzema valors fins a un límit fixat. Quan s'arriba al límit, el *deque* elimina el primer valor perquè pugui entrar-hi un de nou, i així successivament, de manera que es facilita, en el nostre cas, la retroalimentació del *buffer* amb experiències més noves i cada vegada més rellevants.

In [5]:
from collections import namedtuple, deque

Definim el __<i>buffer</i> de repetició d'experiències__:

In [6]:
class experienceReplayBuffer:

    def __init__(self, memory_size=50000, burn_in=10000):
        self.memory_size = memory_size
        self.burn_in = burn_in
        self.buffer = namedtuple('Buffer', 
            field_names=['state', 'action', 'reward', 'done', 'next_state'])
        self.replay_memory = deque(maxlen=memory_size)

    ##Creem una llista d'índexs aleatoris i empaquetem les experiències en <i>arrays<i> de Numpy (això facilita el càlcul posterior de la pèrdua)
    def sample_batch(self, batch_size=32):
        samples = np.random.choice(len(self.replay_memory), batch_size, 
                                   replace=False)
        # Use asterisk operator to unpack deque 
        batch = zip(*[self.replay_memory[i] for i in samples])
        return batch

    ## S'afegeixen les noves experiències 
    def append(self, state, action, reward, done, next_state):
        self.replay_memory.append(
            self.buffer(state, action, reward, done, next_state))

    ## Emplenem el <i>buffer<i> amb experiències aleatòries a l'inici de l'entrenament
    def burn_in_capacity(self):
        return len(self.replay_memory) / self.burn_in

El *burn-in* ens permet emplenar el *buffer* a l'inici de l'entrenament (quan l'agent encara no ha començat a explorar) amb experiències aleatòries perquè estigui prou ple per començar a entrenar amb una varietat d'informació bastant àmplia. 

### 2.2. Definició de l'agent

Una vegada tenim el model definit, només ens queda definir el comportament de l'agent, la manera com aprèn.

Recordem que l'última millora que fèiem a la DQN bàsica i que ens permetia establir l'algorisme DQN final era la introducció d'una **xarxa objectiu**. Amb aquesta segona xarxa (còpia exacta de la principal), calculem el valor objectiu _Q'_, mentre que amb la xarxa principal calculem el valor de _Q_ actual. I cada cert temps se sincronitzen les dues xarxes. Així, aconseguim evitar que l'agent s'estanqui en una regió pel fet que la diferència entre estats (correlació) sigui tan petita que sempre triï la mateixa acció i que acabi per aprendre erròniament. 

Bàsicament, el procés que seguirà l'agent serà el següent:
<ol>
    <li> Emplenar el <i>buffer</i> amb unes quantes experiències aleatòries. </li>
    <li> Interactuar amb l'entorn (fer un pas): 
        <ul>
            <li> Prendre acció segons la probabilitat <i>epsilon</i>. </li>
            <li> Emmagatzemar la informació en el <i>buffer</i>. </li>
            <li> Obtenir la recompensa si està al final de l'episodi en qüestió. </li>
        </ul>
    </li>
    <li> Actualitzar la xarxa neuronal amb la freqüència que s'estableixi i calcular la pèrdua. </li>
    <li> Sincronitzar la xarxa principal amb la xarxa objectiu amb la freqüència que s'estableixi. </li>
    <li> Calcular la mitjana de les recompenses dels últims <i>X</i> episodis (generalment, 100). </li>
    <li> Modificar el valor de <i>epsilon</i> per afavorir l'explotació enfront de l'exploració. </li>
</ol>

L'agent repetirà aquest procés fins que aconsegueixi l'objectiu a partir del qual es considera que ha après a jugar (en **CartPole** és 195, com s'indica en la variable `env.spec.reward_threshold`) o fins que s'esgoti el límit màxim d'episodis establert (hiperparàmetre fixat).

In [7]:
from copy import deepcopy, copy

In [8]:
class DQNAgent:
    
    def __init__(self, env, dnnetwork, buffer, epsilon=0.1, eps_decay=0.99, batch_size=32):
        
        self.env = env
        self.dnnetwork = dnnetwork
        self.target_network = deepcopy(dnnetwork) # xarxa objectiu (còpia de la principal)
        self.buffer = buffer
        self.epsilon = epsilon
        self.eps_decay = eps_decay
        self.batch_size = batch_size
        self.nblock = 100 # bloc dels X últims episodis dels quals es calcularà la mitjana de recompensa
        self.reward_threshold = self.env.spec.reward_threshold # recompensa mitjana a partir de la qual es considera
                                                               # que l'agent ha après a jugar
        self.initialize()
    
    
    def initialize(self):
        self.update_loss = []
        self.training_rewards = []
        self.mean_training_rewards = []
        self.sync_eps = []
        self.total_reward = 0
        self.step_count = 0
        self.state0 = self.env.reset()
        
    
    ## Prenem una nova acció
    def take_step(self, eps, mode='train'):
        if mode == 'explore': 
            # acció aleatòria en el burn-in i en la fase d'exploració (epsilon)
            action = self.env.action_space.sample() 
        else:
            # acció a partir del valor de Q (elecció de l'acció amb millor Q)
            action = self.dnnetwork.get_action(self.state0, eps)
            self.step_count += 1
            
        # Fem l'acció i obtenim el nou estat i la recompensa
        new_state, reward, done, _ = self.env.step(action)
        self.total_reward += reward
        self.buffer.append(self.state0, action, reward, done, new_state) # guardem experiència en el buffer
        self.state0 = new_state.copy()
        
        if done:
            self.state0 = env.reset()
        return done

    
        
    ## Entrenament
    def train(self, gamma=0.99, max_episodes=50000, 
              batch_size=32,
              dnn_update_frequency=4,
              dnn_sync_frequency=2000):
        
        self.gamma = gamma

        # Emplenem el buffer amb N experiències aleatòries ()
        print("Filling replay buffer...")
        while self.buffer.burn_in_capacity() < 1:
            self.take_step(self.epsilon, mode='explore')

            
        episode = 0
        training = True
        print("Training...")
        while training:
            self.state0 = self.env.reset()
            self.total_reward = 0
            gamedone = False
            while gamedone == False:
                # L'agent pren una acció
                gamedone = self.take_step(self.epsilon, mode='train')
               
                # Actualitzem la xarxa principal segons la freqüència establerta
                if self.step_count % dnn_update_frequency == 0:
                    self.update()
                # Sincronitzem la xarxa principal i la xarxa objectiu segons la freqüència establerta
                if self.step_count % dnn_sync_frequency == 0:
                    self.target_network.load_state_dict(
                        self.dnnetwork.state_dict())
                    self.sync_eps.append(episode)
                    
                
                if gamedone:                   
                    episode += 1
                    self.training_rewards.append(self.total_reward) # guardem les recompenses obtingudes
                    self.update_loss = []
                    mean_rewards = np.mean(   # calculem la mitjana de recompensa dels últims X episodis
                        self.training_rewards[-self.nblock:])
                    self.mean_training_rewards.append(mean_rewards)

                    print("\rEpisode {:d} Mean Rewards {:.2f} Epsilon {}\t\t".format(
                        episode, mean_rewards, self.epsilon), end="")
                    
                    # Comprovem que encara queden episodis
                    if episode >= max_episodes:
                        training = False
                        print('\nEpisode limit reached.')
                        break
                    
                    # Acaba el joc si la mitjana de recompenses ha arribat al llindar fixat per a aquest joc  
                    if mean_rewards >= self.reward_threshold:
                        training = False
                        print('\nEnvironment solved in {} episodes!'.format(
                            episode))
                        break
                    
                    # Actualitzem epsilon segons la velocitat de decaïment fixada
                    self.epsilon = max(self.epsilon * self.eps_decay, 0.01)
                    
                
    ## Càlcul de la pèrdua                   
    def calculate_loss(self, batch):
        # Separem les variables de l'experiència i les convertim en tensors 
        states, actions, rewards, dones, next_states = [i for i in batch] 
        rewards_vals = torch.FloatTensor(rewards).to(device=self.dnnetwork.device) 
        actions_vals = torch.LongTensor(np.array(actions)).reshape(-1,1).to(
            device=self.dnnetwork.device)
        dones_t = torch.ByteTensor(dones).to(device=self.dnnetwork.device)
        
        # Obtenim els valors de Q de la xarxa principal
        qvals = torch.gather(self.dnnetwork.get_qvals(states), 1, actions_vals)
        # Obtenim els valors de Q objectiu. El paràmetre detach() evita que aquests valors actualitzin la xarxa objectiu
        qvals_next = torch.max(self.target_network.get_qvals(next_states),
                               dim=-1)[0].detach()
        qvals_next[dones_t] = 0 # 0 en estats terminals
        
        # Calculem l'equació de Bellman
        expected_qvals = self.gamma * qvals_next + rewards_vals
        
        # Calculem la pèrdua
        loss = torch.nn.MSELoss()(qvals, expected_qvals.reshape(-1,1))
        return loss
    

    
    def update(self):
        self.dnnetwork.optimizer.zero_grad()  # eliminem qualsevol gradient passat
        batch = self.buffer.sample_batch(batch_size=self.batch_size) # seleccionem un conjunt del <i>buffer<i>
        loss = self.calculate_loss(batch) # calculem la pèrdua
        loss.backward() # fem la diferència per obtenir els gradients
        self.dnnetwork.optimizer.step() # apliquem els gradients a la xarxa neuronal
        # Guardem els valors de pèrdua
        if self.dnnetwork.device == 'cuda':
            self.update_loss.append(loss.detach().cpu().numpy())
        else:
            self.update_loss.append(loss.detach().numpy())
            


    def plot_rewards(self):
        plt.figure(figsize=(12,8))
        plt.plot(self.training_rewards, label='Rewards')
        plt.plot(self.mean_training_rewards, label='Mean Rewards')
        plt.axhline(self.reward_threshold, color='r', label="Reward threshold")
        plt.xlabel('Episodes')
        plt.ylabel('Rewards')
        plt.legend(loc="upper left")
        plt.show()


### 2.3. Hiperparàmetres

Fixem els hiperparàmetres necessaris:

In [9]:
lr = 0.001            #Velocitat d'aprenentatge
MEMORY_SIZE = 100000  #Màxima capacitat del buffer
MAX_EPISODES = 5000   #Nombre màxim d'episodis (l'agent ha d'aprendre abans d'arribar a aquest valor)
EPSILON = 1           #Valor inicial d'epsilon
EPSILON_DECAY = .99   #Decaïment d'epsilon
GAMMA = 0.99          #Valor gamma de l'equació de Bellman
BATCH_SIZE = 32       #Conjunt a agafar del buffer per a la xarxa neuronal
BURN_IN = 1000        #Nombre d'episodis inicials usats per emplenar el buffer abans d'entrenar
DNN_UPD = 1           #Freqüència d'actualització de la xarxa neuronal 
DNN_SYNC = 2500       #Freqüència de sincronització de pesos entre la xarxa neuronal i la xarxa objectiu

### 2.4. Entrenament

Creem el *buffer* de repetició d'experiències:

In [10]:
buffer = experienceReplayBuffer(memory_size=MEMORY_SIZE, burn_in=BURN_IN)

Carreguem el model de xarxa neuronal:

In [11]:
dqn = DQN(env, learning_rate=lr)

Creem el nostre agent:

In [12]:
agent = DQNAgent(env, dqn, buffer, EPSILON, EPSILON_DECAY, BATCH_SIZE)

Entrenem l'agent amb els hiperparàmetres establerts:

In [ ]:
agent.train(gamma=GAMMA, max_episodes=MAX_EPISODES, 
              batch_size=BATCH_SIZE, dnn_update_frequency=DNN_UPD, dnn_sync_frequency=DNN_SYNC)

### 2.5. Representació de l'aprenentatge de l'agent

In [ ]:
agent.plot_rewards()